In [ ]:
%matplotlib inline

<div class="document" id="histogram-of-oriented-gradients">
<h1 class="title">Histogram of Oriented Gradients</h1>
<p>The <a class="reference external" href="http://en.wikipedia.org/wiki/Histogram_of_oriented_gradients">Histogram of Oriented Gradient</a> (HOG) feature
descriptor [1] is popular for object detection.</p>
<p>In the following example, we compute the HOG descriptor and display
a visualisation.</p>
<div class="section" id="algorithm-overview">
<h1>Algorithm overview</h1>
<p>Compute a Histogram of Oriented Gradients (HOG) by</p>
<blockquote>
<ol class="arabic simple">
<li>(optional) global image normalisation</li>
<li>computing the gradient image in x and y</li>
<li>computing gradient histograms</li>
<li>normalising across blocks</li>
<li>flattening into a feature vector</li>
</ol>
</blockquote>
<p>The first stage applies an optional global image normalisation
equalisation that is designed to reduce the influence of illumination
effects. In practice we use gamma (power law) compression, either
computing the square root or the log of each colour channel.
Image texture strength is typically proportional to the local surface
illumination so this compression helps to reduce the effects of local
shadowing and illumination variations.</p>
<p>The second stage computes first order image gradients. These capture
contour, silhouette and some texture information, while providing
further resistance to illumination variations. The locally dominant
colour channel is used, which provides colour invariance to a large
extent. Variant methods may also include second order image derivatives,
which act as primitive bar detectors - a useful feature for capturing,
e.g. bar like structures in bicycles and limbs in humans.</p>
<p>The third stage aims to produce an encoding that is sensitive to
local image content while remaining resistant to small changes in
pose or appearance. The adopted method pools gradient orientation
information locally in the same way as the SIFT [2]
feature. The image window is divided into small spatial regions,
called &quot;cells&quot;. For each cell we accumulate a local 1-D histogram
of gradient or edge orientations over all the pixels in the
cell. This combined cell-level 1-D histogram forms the basic
&quot;orientation histogram&quot; representation. Each orientation histogram
divides the gradient angle range into a fixed number of
predetermined bins. The gradient magnitudes of the pixels in the
cell are used to vote into the orientation histogram.</p>
<p>The fourth stage computes normalisation, which takes local groups of
cells and contrast normalises their overall responses before passing
to next stage. Normalisation introduces better invariance to illumination,
shadowing, and edge contrast. It is performed by accumulating a measure
of local histogram &quot;energy&quot; over local groups of cells that we call
&quot;blocks&quot;. The result is used to normalise each cell in the block.
Typically each individual cell is shared between several blocks, but
its normalisations are block dependent and thus different. The cell
thus appears several times in the final output vector with different
normalisations. This may seem redundant but it improves the performance.
We refer to the normalised block descriptors as Histogram of Oriented
Gradient (HOG) descriptors.</p>
<p>The final step collects the HOG descriptors from all blocks of a dense
overlapping grid of blocks covering the detection window into a combined
feature vector for use in the window classifier.</p>
</div>
<div class="section" id="references">
<h1>References</h1>
<table class="docutils footnote" frame="void" id="id1" rules="none">
<colgroup><col class="label" /><col /></colgroup>
<tbody valign="top">
<tr><td class="label">[1]</td><td>Dalal, N. and Triggs, B., &quot;Histograms of Oriented Gradients for
Human Detection,&quot; IEEE Computer Society Conference on Computer
Vision and Pattern Recognition, 2005, San Diego, CA, USA.</td></tr>
</tbody>
</table>
<table class="docutils footnote" frame="void" id="id2" rules="none">
<colgroup><col class="label" /><col /></colgroup>
<tbody valign="top">
<tr><td class="label">[2]</td><td>David G. Lowe, &quot;Distinctive image features from scale-invariant
keypoints,&quot; International Journal of Computer Vision, 60, 2 (2004),
pp. 91-110.</td></tr>
</tbody>
</table>
</div>
</div>


In [2]:
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, color, exposure


image = color.rgb2gray(data.astronaut())

fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualise=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 0.02))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()
